In [17]:
import torch
import torchvision.transforms as transforms
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
import cv2

# Load the pre-trained SSD model with MobileNetV3 backbone
model = ssdlite320_mobilenet_v3_large(pretrained=True)
model.eval()

# Define a function to detect faces in a frame
def detect_faces(frame, model):
    transform = transforms.Compose([transforms.ToTensor()])
    image = transform(frame).unsqueeze(0)

    with torch.no_grad():
        prediction = model(image)

    boxes = prediction[0]['boxes'].cpu().numpy()
    scores = prediction[0]['scores'].cpu().numpy()

    # Filter out low-confidence detections
    threshold = 0.8
    detected_faces = [box for box, score in zip(boxes, scores) if score > threshold]

    return detected_faces

# Open the video capture
cap = cv2.VideoCapture('C:\\Users\\Ameyo\\Downloads\\faces01 (1) (1).mp4')

# Get the video's frames per second (fps) and frame dimensions
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object to save the edited video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

# Initialize counters
total_faces = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detected_faces = detect_faces(frame, model)
    num_detected_faces = len(detected_faces)
    total_faces += num_detected_faces  # Update total faces count

    # Display the number of detected faces and total faces in the top right corner
    text = f'Face: {num_detected_faces},Total: {total_faces}'
    cv2.putText(frame, text, (width - 350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    for box in detected_faces:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

# Display the total number of faces detected in the entire video on the next line
print(f'Total Faces Detected in Video: {total_faces}\n')



error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
